# 【Q13】What is the percentage of female researcher in your group ?

- 【Q13】What is the percentage of female researcher in your group ?
- 【Q14】What do you think about the percentage above ?

In [ ]:
import altair as alt
import pandas as pd
import scipy as sp
import titanite as ti

print(f"Altair {alt.__version__}")
print(f"Pandas {pd.__version__}")
print(f"SciPy {sp.__version__}")
print(f"Titanite {ti.__version__}")

データを読み込む

In [ ]:
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
d = ti.Data(read_from=f_csv, load_from=f_cfg)
data = d.read()
# data.info()

In [ ]:
# x = ["q13", "q13_binned"]
# x = ["q13_binned", "q14"]
x = ["q14"]
headers = d.crosstab_headers(x, list(data.columns))

In [ ]:
cross_tabs, heatmaps, chi2_data = ti.core.crosstab_loop(data, headers)
chi2_data

In [ ]:
mark = (
    alt.Chart(chi2_data)
    .mark_point()
    .encode(
        alt.X("p_value"),
        alt.Y("questions"),
        tooltip=["questions", "p_value", "dof", "statistic"],
    )
)

rule = alt.Chart().mark_rule(color="orange").encode(x=alt.datum(0.05))

(mark + rule).properties(
    width=500,
)

``p < 0.05``の項目を調べる

In [ ]:
p005 = chi2_data.query("p_value < 0.05")
p005

In [ ]:
for name, hm in heatmaps.items():
    if name in list(p005.questions):
        hm.properties(
            title=name,
            width=400,
            height=400,
        ).display()

# q13とq14の相関の詳細を確認中

- q13のビン幅（5%）でよさそうかを確認？

In [ ]:
range_x = alt.Scale(domain=[-10, 100])
base = (
    alt.Chart(data)
    .encode(
        alt.Y("q14"),
        alt.Color("count()").scale(scheme="blues"),
    )
    .properties(
        width=800,
        height=400,
    )
)

base0 = base.encode(alt.X("q13").scale(range_x)).properties(title="ビン幅：なし")
base5 = base.encode(alt.X("q13").bin(step=5).scale(range_x)).properties(
    title="ビン幅：5%"
)
base10 = base.encode(alt.X("q13").bin(step=10).scale(range_x)).properties(
    title="ビン幅：10%"
)

In [ ]:
mark = base0.mark_point()
text = base0.mark_text(dy=-20).encode(alt.Text("count()"))
bin0 = mark + text
# bin0

In [ ]:
mark = base5.mark_rect()
text = base5.mark_text(dy=-20, color="black").encode(alt.Text("count()"))
bin5 = mark + text
# bin5

In [ ]:
mark = base10.mark_rect()
text = base10.mark_text(dy=-20).encode(alt.Text("count()"))
bin10 = mark + text
# bin10

In [ ]:
bin0 & bin5 & bin10

``q13``（``q13_binned``）と``q14``のジェンダー比率

In [ ]:
base = (
    alt.Chart(data)
    .encode(
        alt.X("q13_binned"),
        alt.Y("count()").stack("normalize"),
        alt.Color("q02:N"),
        alt.Order("count()"),
    )
    .properties(
        height=200,
        width=400,
    )
)

mark = base.mark_bar(opacity=0.5)
text = base.mark_text(dy=10).encode(
    alt.Text("count()"),
)
top = mark + text

In [ ]:
base = (
    alt.Chart(data)
    .encode(
        alt.Y("q14"),
        alt.X("count()").stack("normalize"),
        alt.Color("q02:N"),
        alt.Order("count()"),
    )
    .properties(
        height=400,
        width=200,
    )
)

mark = base.mark_bar(opacity=0.5)
text = base.mark_text(dx=-10).encode(alt.Text("count()"))
right = mark + text

In [ ]:
base = (
    alt.Chart(data)
    .encode(
        alt.X("q13_binned"),
        alt.Y("q14"),
    )
    .properties(
        width=400,
        height=400,
    )
)

mark = base.mark_rect().encode(
    alt.Color("count()").scale(scheme="blues"),
)

text = base.mark_text().encode(alt.Text("count()"))

main = mark + text

In [ ]:
top & (main | right)